In [1]:
# Import Packages
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Lambda, Cropping2D, Convolution2D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
samples = []
with open('./drivesimulator_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [2]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './drivesimulator_data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [9]:
image = cv2.imread('./drivesimulator_data/IMG/' + train_samples[0][0].split('/')[-1])
image_shape = np.array(image).shape
print (image_shape)

(160, 320, 3)


In [11]:
# hyper parameter
top = 50
down = 20
left= 0
right = 0

In [12]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=image_shape))
model.add(Cropping2D(cropping = ((top, down), (left, right)), input_shape= image_shape)) 
model.add(Convolution2D(24, 5, 5, subsample = (2, 2), activation = 'relu'))     # Convolution layer 1: Output 32x156x24
model.add(Convolution2D(36, 5, 5, subsample = (2, 2), activation = 'relu'))     # Convolution layer 2: Output 14x76x36
model.add(Convolution2D(48, 5, 5, subsample = (2, 2), activation = 'relu'))     # Convolution layer 3: Output 5x36x48
model.add(Convolution2D(64, 3, 3, activation = 'relu'))                         # Convolution layer 4: Output 3x34x64
model.add(Convolution2D(64, 3, 3, activation = 'relu'))                         # Convolution layer 5: Output 1x32x64
model.add(Flatten())                                                            # Flatten: Output 2048
model.add(Dense(100, activation = 'relu'))                                      # Fully connected layer 1: Output 100
model.add(Dense(50, activation = 'relu'))                                       # Fully connected layer 2: Output 50
model.add(Dense(10, activation = 'relu'))                                       # Fully connected layer 3: Output 10
model.add(Dense(1))  

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=3)

Epoch 1/3


/home/lei/.local/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/home/lei/.local/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1859, validation_data=<generator..., epochs=3, validation_steps=465)`
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lei/.local/lib/python3.5/site-packages/keras/engine/training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
  File 

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None